In [2]:
#preprocessing libriaris
import nltk
nltk.download("stopwords")
#stopwords modules
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function (stopwords, puncuation, etc)
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

#Examples    
preprocess_text("<|startoftext|>Выкинь все, что есть в твоих мыслях: грусть, пустоту, печаль. Помнишь, как ты подарил мне лето, в то время, как был февраль.<|endoftext|>")

'<| startoftext |> выкидывать твой мысль грусть пустота печаль помнить подарить лето время февраль <| endoftext |>\n'

In [4]:
import pandas as pd
import numpy as np
import io

qoute_path = '../parsing/data/cleaned_tagged.txt'

df = pd.read_csv(qoute_path, sep='<\d*>', engine ='python', names=['quotes']) #reading quotes to the dataframe
with open(qoute_path, 'r', encoding='utf-8') as file:
    data = file.read().replace('\n', ' nextquote ')
    processed_data = preprocess_text(data)  #preprocessing qoutes
#dataframe with quotes and their processed versions
df.insert(loc=1, column="processed_quotes", value=processed_data.split('nextquote')[:-1], allow_duplicates=True) 

In [11]:
df

,quotes,processed_quotes
0,"Выкинь все, что есть в твоих мыслях: грусть, п...",выкидывать твой мысль грусть пустота печаль по...
1,"Порой молчание, красноречивее любых произнесен...",пора молчание красноречиво любой произносить ...
2,"А вы знаете, жизнь ведь замечательна и удивите...",знать жизнь замечательный удивительный услови...
3,- А че он такой красивый?,че красивый
4,"Не тот велик, кто никогда не падал, а тот вели...",большой падать большой падать вставать
...,...,...
12522,Величайшая твердость и есть величайшее милосер...,великий твердость великий милосердие
12523,Ничто великое в мире не совершается без страсти.,ничто великий мир совершаться страсть
12524,Бог рад нечетному числу.,бог рад нечетный число
12525,"Иные утверждают, что душа есть воздух.",иной утверждать душа воздух


In [14]:
quote_tokens = [q.split() for q in df['processed_quotes'].values.tolist()]
quote_words = [q.split() for q in df['quotes'].values.tolist()]
for 
#print(quote_tokens[0])
#print(quote_words[0])

['выкидывать', 'твой', 'мысль', 'грусть', 'пустота', 'печаль', 'помнить', 'подарить', 'лето', 'время', 'февраль']
['Выкинь', 'все,', 'что', 'есть', 'в', 'твоих', 'мыслях:', 'грусть,', 'пустоту,', 'печаль.', 'Помнишь,', 'как', 'ты', 'подарил', 'мне', 'лето,', 'в', 'то', 'время,', 'как', 'был', 'февраль.']


In [33]:
from gensim.models import Word2Vec
# define a model with parameters
# and train it on our data
model = Word2Vec(quote_tokens, # data for model to train on
                 vector_size=32,         # embedding vector size
                 min_count=5,     # consider words that occured at least 5 times
                 window=3,
                 hs = 1).wv     # define context as a 3-word window around the target word

In [31]:
wordic ='мама'
model.get_vector(wordic)
model.most_similar(wordic)

print(len(model.index_to_key))

2680


In [32]:
n_words = 1000
words = model.index_to_key
word_vectors = [model.get_vector(word) for word in words]

In [23]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca = PCA(n_components=32)
pca.fit(word_vectors)
word_vectors_pca = pca.transform(word_vectors)
ss = StandardScaler().fit(word_vectors_pca)
word_vectors_pca = ss.transform(word_vectors_pca)

In [24]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig


draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

Loading BokehJS ...

Figure(id='1004', ...)

In [28]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, verbose=32)
#tsne.fit(word_vectors)
word_vectors_tsne = tsne.fit_transform(word_vectors)

ss = StandardScaler().fit(word_vectors_tsne)
word_vectors_tsne = ss.transform(word_vectors_tsne)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.001s...
[t-SNE] Computed neighbors for 1000 samples in 0.032s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 0.054294
[t-SNE] Computed conditional probabilities in 0.050s
[t-SNE] Iteration 50: error = 60.6410446, gradient norm = 0.3141488 (50 iterations in 10.512s)
[t-SNE] Iteration 100: error = 56.7306557, gradient norm = 0.2477854 (50 iterations in 9.714s)
[t-SNE] Iteration 150: error = 56.0499115, gradient norm = 0.2580498 (50 iterations in 10.091s)
[t-SNE] Iteration 200: error = 55.3717346, gradient norm = 0.2453297 (50 iterations in 9.621s)
[t-SNE] Iteration 250: error = 54.8031044, gradient norm = 0.2608456 (50 iterations in 9.518s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 54.803104
[t-SNE] Iteration 300: error = 0.8770762, gradient norm = 0.0010110 (50 iterations in 9.120s)
[t-SNE] Iteration 350: error = 0.8026948, gradient norm = 0.000

In [30]:
output_notebook()
draw_vectors(word_vectors_tsne[:, 0], word_vectors_tsne[:, 1], token=words)

Loading BokehJS ...

Figure(id='1115', ...)